In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

In [ ]:
path = "/content/drive/MyDrive/test"

In [ ]:
import os

num_skipped = 0
for folder_name in ("Bed", "Chair", "Sofa"):
    folder_path = os.path.join(path, folder_name)
    print('---', folder_path, '---')
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

--- /content/drive/MyDrive/test/Bed ---
--- /content/drive/MyDrive/test/Chair ---
--- /content/drive/MyDrive/test/Sofa ---
Deleted 0 images


In [ ]:
image_size = (180, 180)
batch_size = 32

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="both",
    seed=1337,
    label_mode = "categorical",
    image_size=image_size,
    batch_size=batch_size,
)

Found 300 files belonging to 3 classes.
Using 240 files for training.
Using 60 files for validation.


In [ ]:
'''
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(labels[i])
        plt.axis("off")
'''

print('--')

--


In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [ ]:
augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = keras.Sequential([
            
    layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape = (180, 180, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(filters=64,kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(3, activation='softmax')
])

In [ ]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5", 
                                    "val_loss", 
                                    save_best_only=True),
]

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/50
8/8 [==============================] - 23s 491ms/step - loss: 331.3553 - accuracy: 0.3417 - val_loss: 12.2021 - val_accuracy: 0.3167
Epoch 2/50
8/8 [==============================] - 6s 224ms/step - loss: 16.0636 - accuracy: 0.3625 - val_loss: 1.9411 - val_accuracy: 0.3167
Epoch 3/50
8/8 [==============================] - 4s 218ms/step - loss: 4.4023 - accuracy: 0.4125 - val_loss: 1.0051 - val_accuracy: 0.3500
Epoch 4/50
8/8 [==============================] - 6s 456ms/step - loss: 1.8586 - accuracy: 0.3917 - val_loss: 1.4379 - val_accuracy: 0.3000
Epoch 5/50
8/8 [==============================] - 4s 203ms/step - loss: 1.1614 - accuracy: 0.3375 - val_loss: 1.2361 - val_accuracy: 0.3500
Epoch 6/50
8/8 [==============================] - 4s 214ms/step - loss: 0.9494 - accuracy: 0.5542 - val_loss: 1.0693 - val_accuracy: 0.4333
Epoch 7/50
8/8 [==============================] - 6s 223ms/step - loss: 0.9018 - accuracy: 0.5292 - val_loss: 0.9149 - val_accuracy: 0.5167
Epoch 8/50
8/8 

In [ ]:
loaded_model = tf.keras.models.load_model('/content/save_at_24.h5')

In [ ]:
!pip install np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def classify(path):
  image_size = (180, 180)

  img = keras.preprocessing.image.load_img(
      path, target_size=image_size
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # create batch axis

  classes = {0: 'Bed', 1: 'Chair', 2: 'Sofa'}
  prediction = classes[np.argmax(loaded_model.predict(img_array))]

  return prediction

In [ ]:
classify("/content/drive/MyDrive/test/Bed/ALBI BED.jpg")

1/1 [==============================] - 0s 22ms/step


'Bed'